In [1]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time
import math
import tempfile
from functools import partial

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

from mlprimitives.custom.timeseries_preprocessing import time_segments_aggregate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from mlprimitives.custom.timeseries_preprocessing import rolling_window_sequences
from orion.primitives.timeseries_preprocessing import slice_array_by_dims
from mlprimitives import load_primitive
import numpy as np
from orion.primitives.tadgan import TadGAN

from tensorflow.keras import backend as K
from keras.layers.merge import _Merge

logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

Using TensorFlow backend.


# Positional encoding

In [2]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [3]:
# Test Example
n, d = 100, 25
pos_encoding = positional_encoding(n, d)
print(pos_encoding.shape)
pos_encoding = pos_encoding[0]

(1, 100, 25)


# Masking

In [4]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [5]:
# 0 are pads
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor 'strided_slice_1:0' shape=(3, 1, 1, 5) dtype=float32>

In [6]:
# look-ahead mask
x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

<tf.Tensor 'sub:0' shape=(3, 3) dtype=float32>

# Multihead Attention

In [7]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights




class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [8]:
temp_mha = MultiHeadAttention(d_model=25, num_heads=5)
y = tf.random.uniform((1, 100, 25))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

(TensorShape([Dimension(1), Dimension(100), Dimension(25)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(100), Dimension(100)]))

# Point wise feed forward network

In [9]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [10]:
sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

TensorShape([Dimension(64), Dimension(50), Dimension(512)])

# Encoder

In [11]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [12]:
sample_encoder_layer = EncoderLayer(512, 8, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, 512)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

TensorShape([Dimension(64), Dimension(43), Dimension(512)])

In [13]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    # self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    # x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
   

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [14]:
sample_encoder = Encoder(num_layers=2, d_model=25, num_heads=5,
                         dff=1, input_vocab_size=None,
                         maximum_position_encoding=10000)
temp_input = tf.random.uniform((1, 100, 25), dtype=tf.float32, minval=0, maxval=200)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

(1, 100, 25)


In [15]:
sample_encoder.trainable

True

# Decoder

In [16]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [17]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output,
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

TensorShape([Dimension(64), Dimension(50), Dimension(512)])

In [18]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [19]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((64, 26), dtype=tf.float32, minval=0, maxval=200)

output, attn = sample_decoder(temp_input,
                              enc_output=sample_encoder_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

ValueError: in converted code:

    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_48479/955386546.py:29 call  *
        x, block1, block2 = self.dec_layers[i](x, enc_output, training,
    /Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:854 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_48479/1969818830.py:26 call  *
        attn2, attn_weights_block2 = self.mha2(
    /Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:854 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_48479/3292487066.py:71 call  *
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_48479/3292487066.py:60 spli

# Transformer

In [20]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    inp, tar = inputs

    enc_padding_mask, look_ahead_mask, dec_padding_mask = self.create_masks(inp, tar)

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

  def create_masks(self, inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, look_ahead_mask, dec_padding_mask

In [21]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048,
    input_vocab_size=8500, target_vocab_size=8000,
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 38), dtype=tf.float32, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.float32, minval=0, maxval=200)

fn_out, _ = sample_transformer([temp_input, temp_target], training=False)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

ValueError: in converted code:

    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_48479/459540881.py:19 call  *
        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    /Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:854 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_48479/1886624111.py:25 call  *
        x += self.pos_encoding[:, :seq_len, :]
    /Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/tensorflow_core/python/ops/math_ops.py:899 binary_op_wrapper
        return func(x, y, name=name)
    /Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/tensorflow_core/python/ops/math_ops.py:1197 _add_dispatch
        return gen_math_ops.add_v2(x, y, name=name)
    /Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/te

# TadGan

In [22]:
X = np.load('processed/data/X.npy')
target = np.load('processed/data/y.npy')
X.shape, y.shape

((2718, 100, 25), TensorShape([Dimension(1), Dimension(100), Dimension(25)]))

In [23]:
input_shape=(100, 25)
target_shape=(100, 1)
latent_dim=20
learning_rate=0.0005
epochs=70
batch_size=64
iterations_critic=5
latent_shape = (latent_dim, 1)

shape = np.asarray(X)[0].shape
length = shape[0]
target_shape = np.asarray(target)[0].shape


generator_reshape_dim =  length // 2
generator_reshape_shape = (length // 2, 1)
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape



lstm_units = 100
dense_units = 20

In [28]:
def build_encoder(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(Encoder(num_layers=2, d_model=25, num_heads=5,
                      dff=1, input_vocab_size=None,
                      maximum_position_encoding=10000))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.Reshape(target_shape=encoder_reshape_shape))
    return tf.keras.Model(x, model(x))
              
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100, 25)]         0         
_________________________________________________________________
sequential_17 (Sequential)   (None, 20, 1)             55572     
Total params: 55,572
Trainable params: 55,572
Non-trainable params: 0
_________________________________________________________________


In [29]:
def build_generator(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=generator_reshape_dim))
    model.add(tf.keras.layers.Reshape(target_shape=generator_reshape_shape))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(tf.keras.layers.UpSampling1D(size=2))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1)))
    model.add(tf.keras.layers.Activation(activation='tanh'))
    return tf.keras.Model(x, model(x))
              
generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
sequential_20 (Sequential)   (None, 100, 1)            133787    
Total params: 133,787
Trainable params: 133,787
Non-trainable params: 0
_________________________________________________________________


In [30]:
def build_critic_x(input_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_x = build_critic_x(
    input_shape=critic_x_input_shape
)
critic_x.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100, 1)]          0         
_________________________________________________________________
sequential_21 (Sequential)   (None, 1)                 67393     
Total params: 67,393
Trainable params: 67,393
Non-trainable params: 0
_________________________________________________________________


In [31]:
def build_critic_z(input_shape, dense_units=20):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)
critic_z.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
sequential_22 (Sequential)   (None, 1)                 861       
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [32]:
class RandomWeightedAverage(_Merge):
    def _merge_function(self, inputs):
        """
        Args:
            inputs[0] x     original input
            inputs[1] x_    predicted input
        """
        alpha = K.random_uniform((64, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [42]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [35]:
generator.trainable = False
encoder.trainable = False

x = tf.keras.Input(shape=input_shape)
y = tf.keras.Input(shape=target_shape)
z = tf.keras.Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_) # Fake
valid_x = critic_x(y) # Truth

In [43]:
# interpolated_x = RandomWeightedAverage()([y, x_])
alpha = K.random_uniform((64, 1, 1))
interpolated_x = (alpha * [y, x_][0]) + ((1 - alpha) * [y, x_][1])
validity_interpolated_x = critic_x(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_model = tf.keras.Model(inputs=[y, z], outputs=[valid_x, fake_x,validity_interpolated_x])
critic_x_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x], 
                       optimizer=optimizer, loss_weights=[1, 1, 10])

In [46]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
# interpolated_z = RandomWeightedAverage()([z, z_])
alpha = K.random_uniform((64, 1, 1))
interpolated_z = (alpha * [z, z_][0]) + ((1 - alpha) * [z, z_][1])
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(_gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = tf.keras.Model(inputs=[x, z], outputs=[valid_z, fake_z,validity_interpolated_z])
critic_z_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,
                                  partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])

In [47]:
critic_x.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

z_gen = tf.keras.Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = tf.keras.Input(shape=input_shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x(x_gen_)
fake_gen_z = critic_z(z_gen_)

encoder_generator_model = tf.keras.Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,'mse'], 
                                optimizer=optimizer,
                                loss_weights=[1, 1, 10])

In [ ]:
fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

indices = np.arange(X.shape[0])
for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X[indices]
    y_ = target[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_cx_loss.append(
                critic_x_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))

        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))

    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cz_loss, g_loss))

Epoch: 1/70, [Dx loss: [-4.24187    -8.015909    0.9574412   0.28165972]] [Dz loss: [ 4.5781426  -0.99709797  4.449582    0.11256589]] [G loss: [ 1.4436641  -0.94184685 -3.9820216   0.6367533 ]]
Epoch: 2/70, [Dx loss: [-6.056184   -8.349213    1.2994711   0.09935593]] [Dz loss: [ 2.772599   -1.0329262   2.2865674   0.15189575]] [G loss: [ 5.2627206  -0.92800975  1.0047191   0.5186012 ]]
Epoch: 3/70, [Dx loss: [-5.64261    -3.8386726  -2.7562244   0.09522863]] [Dz loss: [17.605806   -0.9500634  13.202464    0.53534067]] [G loss: [ 0.49423003  2.6964743  -7.4496913   0.52474475]]
Epoch: 4/70, [Dx loss: [-5.664611   -6.413623   -0.14579882  0.08948103]] [Dz loss: [34.441914   -0.69006264 33.09275     0.20392092]] [G loss: [-2.0092443e+01 -2.2853605e-02 -2.4544413e+01  4.4748238e-01]]
Epoch: 5/70, [Dx loss: [-5.907487   -7.4440904   0.5737573   0.09628473]] [Dz loss: [-4.9435816  -0.5135335  -5.332631    0.09025834]] [G loss: [18.284224   -0.63521993 14.582884    0.43365592]]
Epoch: 6/70, 